# data_d25a.ipynb
1. For GMSL and RSL at gauges, save fusion, high-end, low-end, and central projections for 2020–2100, and also gauge info.
2. For cities, save city info, gauge info, high-end, low-end, and central projections for 2100 (if gauge nearby).

Author: Benjamin S. Grandey.

In [1]:
import d25a
import datetime
import numpy as np
import pandas as pd
import xarray as xr

In [2]:
# Get start datetime
start_dt = datetime.datetime.now()

In [3]:
# Print package versions
print(d25a.get_watermark())

Python implementation: CPython
Python version       : 3.10.16
IPython version      : 8.31.0

matplotlib: 3.10.0
numpy     : 2.2.2
pandas    : 2.2.3
seaborn   : 0.13.2
xarray    : 2025.1.1

conda environment: d25a-rsl-fusion

Compiler    : Clang 18.1.8 
OS          : Darwin
Release     : 22.6.0
Machine     : arm64
Processor   : arm
CPU cores   : 8
Architecture: 64bit



In [4]:
# Make output directories if they don't exist
for data_dir in (d25a.DATA_DIR, d25a.DATA_DIR / 'gmsl', d25a.DATA_DIR / 'gauges', d25a.DATA_DIR / 'cities'):
    if not data_dir.exists():
        data_dir.mkdir()

## 1. GMSL and RSL at gauges

### 1a. Identify gauges with missing RSL data
These gauges will be dropped.

In [5]:
# Read fusion RSL data for one scenario
qfs_da = d25a.get_sl_qfs(workflow='fusion_1e+2e', gmsl_rsl_novlm='rsl', scenario='ssp585').copy()
# Identify locations with missing data
missing_gauges = qfs_da.where(qfs_da.isnull(), drop=True).locations.data
# Print some information about these gauges
print(f'{len(missing_gauges)} gauges have missing RSL data:')
for gauge_id in missing_gauges:
    gauge_info = d25a.get_gauge_info(gauge=gauge_id)
    print(f"{gauge_id}, {gauge_info['gauge_name']}, {gauge_info['country']}")

14 gauges have missing RSL data:
126, TROIS-RIVIERES, CANADA
137, PORT-SAINT-FRANCOIS, CANADA
144, BATISCAN, CANADA
173, QUEBEC, CANADA
192, NEUVILLE, CANADA
201, DESCHAILLONS, CANADA
387, GRONDINES, CANADA
951, PORTNEUF, CANADA
999, ST-FRANCOIS, CANADA
1005, CHAMPLAIN, CANADA
1219, TADOUSSAC, CANADA
1244, ST-JOSEPH-DE-LA-RIVE, CANADA
1392, PORT-ALFRED, CANADA
1798, BECANCOUR, CANADA


### 1b. Save fusion, high-end, and low-end projections

In [6]:
# Loop over GMSL, RSL, and RSL without VLM component
for gmsl_rsl_novlm in ('gmsl', 'rsl', 'novlm'):
    # Output directory
    if gmsl_rsl_novlm == 'gmsl':
        out_dir = d25a.DATA_DIR / 'gmsl'
    else:
        out_dir = d25a.DATA_DIR / 'gauges'
    # Loop over two scenarios
    for scenario in ['ssp585', 'ssp126']:
        # Derive fusion projection
        qfs_da = d25a.get_sl_qfs(workflow='fusion_1e+2e', gmsl_rsl_novlm=gmsl_rsl_novlm, scenario=scenario).copy()
        # Drop gauges with missing RSL data
        if gmsl_rsl_novlm != 'gmsl':
            for gauge_id in missing_gauges:
                qfs_da.sel(locations=gauge_id).data[:] = np.nan  # this changes novlm data to also be NaN
            qfs_da = qfs_da.dropna(dim='locations')
        # Save fusion projection
        out_fn = out_dir / f'{gmsl_rsl_novlm}_fusion_{scenario}_d25a.nc'
        if gmsl_rsl_novlm == 'gmsl':
            print(f'Writing {out_fn.name}')
        else:
            print(f'Writing {out_fn.name} ({len(qfs_da.locations)} gauges)')
        qfs_da.to_netcdf(out_fn)
        # Derive and save high-end or low-end projection, depending on scenario
        if scenario == 'ssp585':
            high_da = qfs_da.sel(quantiles=0.95).squeeze()
            out_fn = out_dir / f'{gmsl_rsl_novlm}_high_d25a.nc'
            print(f'Writing {out_fn.name}')
            high_da.to_netcdf(out_fn)
        elif scenario == 'ssp126':
            low_da = qfs_da.sel(quantiles=0.05).squeeze()
            out_fn = out_dir / f'{gmsl_rsl_novlm}_low_d25a.nc'
            print(f'Writing {out_fn.name}')
            low_da.to_netcdf(out_fn)

Writing gmsl_fusion_ssp585_d25a.nc
Writing gmsl_high_d25a.nc
Writing gmsl_fusion_ssp126_d25a.nc
Writing gmsl_low_d25a.nc
Writing rsl_fusion_ssp585_d25a.nc (1016 gauges)
Writing rsl_high_d25a.nc
Writing rsl_fusion_ssp126_d25a.nc (1016 gauges)
Writing rsl_low_d25a.nc
Writing novlm_fusion_ssp585_d25a.nc (1016 gauges)
Writing novlm_high_d25a.nc
Writing novlm_fusion_ssp126_d25a.nc (1016 gauges)
Writing novlm_low_d25a.nc


### 1c. Save central projection
Defined as median of medium confidence mean under SSP2-4.5.

In [7]:
# Loop over GMSL/RSL and scenarios
for gmsl_rsl_novlm in ('gmsl', 'rsl', 'novlm'):
    # Output directory
    if gmsl_rsl_novlm == 'gmsl':
        out_dir = d25a.DATA_DIR / 'gmsl'
    else:
        out_dir = d25a.DATA_DIR / 'gauges'
    # Derive medium confidence mean under SSP2-4.5
    qfs_da = d25a.get_sl_qfs(workflow='mean_1e+2e', gmsl_rsl_novlm=gmsl_rsl_novlm, scenario='ssp245').copy()
    # Drop locations with NaN
    if gmsl_rsl_novlm != 'gmsl':
        # Drop gauges with missing RSL data
        if gmsl_rsl_novlm != 'gmsl':
            for gauge_id in missing_gauges:
                qfs_da.sel(locations=gauge_id).data[:] = np.nan  # this changes novlm data to also be NaN
            qfs_da = qfs_da.dropna(dim='locations')
    # Derive and Save central projection
    central_da = qfs_da.sel(quantiles=0.5).squeeze()
    out_fn = out_dir / f'{gmsl_rsl_novlm}_central_d25a.nc'
    print(f'Writing {out_fn.name}')
    central_da.to_netcdf(out_fn)

Writing gmsl_central_d25a.nc
Writing rsl_central_d25a.nc
Writing novlm_central_d25a.nc


### 1d. Save gauge information

In [8]:
# Create DataFrame to hold gauge information
gauge_info_df = pd.DataFrame(columns=['gauge_id', 'gauge_name', 'lat', 'lon', 'country'])
# Loop over locations for which projections are available
qfs_da = d25a.get_sl_qfs().copy()
for location in qfs_da.locations.data:
    if location not in missing_gauges:
        # Get information about this gauge and save to DataFrame
        gauge_info = d25a.get_gauge_info(location)
        gauge_info_df.loc[len(gauge_info_df)] = gauge_info
# Index by gauge_id
gauge_info_df = gauge_info_df.set_index('gauge_id')
# Save to CSV
out_fn = d25a.DATA_DIR / 'gauges' / f'gauge_info_d25a.csv'
print(f'Writing {out_fn.name} ({len(gauge_info_df)} gauges)')
gauge_info_df.to_csv(out_fn)
gauge_info_df.head()

Writing gauge_info_d25a.csv (1016 gauges)


,gauge_name,lat,lon,country
gauge_id,,,,
1,BREST,48.38,-4.49,FRANCE
2,SWINOUJSCIE,53.92,14.23,POLAND
3,SHEERNESS,51.45,0.74,UNITED KINGDOM
5,HOLYHEAD,53.31,-4.62,UNITED KINGDOM
7,CUXHAVEN_2,53.87,8.72,GERMANY


## 2. Data for cities

### 2a. All cities
The cities are urban agglomerations with a population of at least 300,000 inhabitants in 2018, according to the UN's World Urbanization Prospects (2018).

Low-end, central, and high-end RSL projections for 2100 are saved if the distance to the nearest available tide gauge is ≤ 100km.

In [9]:
# Read World Urbanisation Prospects 2018 data
all_df = pd.read_excel('data_in/wup18/WUP2018-F12-Cities_Over_300K.xls', header=16, usecols='A,C,E,G,H,X', index_col=None)
all_df = all_df.rename(columns={'Index': 'city_index', 'Country or area': 'city_country', 'Urban Agglomeration': 'city_name',
                                'Latitude': 'city_lat', 'Longitude': 'city_lon', 2025: 'population_2025_1000s'})
all_df = all_df.set_index('city_index')
# Loop over these cities and find nearest tide gauge and distance
for index, row_ser in all_df.iterrows():
    lat0 = row_ser['city_lat']  # latitude of city
    lon0 = row_ser['city_lon']  # longitude of city
    temp_df = gauge_info_df.copy()  # copy tide gauge data (from above)
    temp_df['distance_km'] = 6378 * np.arccos(  # calculate great-circle distance between city and all available gauges
        np.sin(np.radians(lat0)) * np.sin(np.radians(temp_df['lat'])) +
        np.cos(np.radians(lat0)) * np.cos(np.radians(temp_df['lat'])) * np.cos(np.radians(temp_df['lon'] - lon0)))
    temp_df = temp_df.sort_values(by=['distance_km']).reset_index()  # sort by distance
    temp_df = temp_df.rename(columns={'lat': 'gauge_lat', 'lon': 'gauge_lon'})
    for col in ['gauge_id', 'gauge_name', 'gauge_lat', 'gauge_lon', 'distance_km']:
        all_df.loc[index, col] = temp_df.loc[0, col]  # save gauge info to all_df
all_df['distance_km'] = all_df['distance_km'].round(0).astype(int)  # round to nearest km
all_df['gauge_id'] = all_df['gauge_id'].astype(int)  # gauge_id should also be int
# If distance ≤ 100km, save low-end, central, and high-end rsl and novlm projections for 2100
for rsl_novlm in ('rsl', 'novlm'):  # loop over rsl and novlm
    for low_central_high in ('low', 'central', 'high'):  # loop over low-end, central, and high-end projections
        col = f'{rsl_novlm}_{low_central_high}_2100'  # column name
        gauge_da = xr.open_dataset(d25a.DATA_DIR / 'gauges' / f'{rsl_novlm}_{low_central_high}_d25a.nc'
                                   )['sea_level_change'].sel(years=2100)  # get year-2100 projections at gauges
        for index, row_ser in all_df.iterrows():  # loop over cities
            if row_ser['distance_km'] <= 100:  # check if distance to nearest gauge is ≤ 100km
                all_df.loc[index, col] = float(gauge_da.sel(locations=row_ser['gauge_id']))  # save projection to all_df
# Save to CSV
out_fn = d25a.DATA_DIR / 'cities' / f'all_cities_d25a.csv'
print(f'Writing {out_fn.name} ({len(all_df)} cities; {len(all_df.dropna())} within 100km of gauge)')
all_df.to_csv(out_fn)
all_df.head(30)

Writing all_cities_d25a.csv (1860 cities; 430 within 100km of gauge)


,city_country,city_name,city_lat,city_lon,population_2025_1000s,gauge_id,gauge_name,gauge_lat,gauge_lon,distance_km,rsl_low_2100,rsl_central_2100,rsl_high_2100,novlm_low_2100,novlm_central_2100,novlm_high_2100
city_index,,,,,,,,,,,,,,,,
1,Afghanistan,Herat,34.348170,62.199670,752.910,204,KARACHI,24.81,66.97,1157,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,Kabul,34.528887,69.172460,4877.024,204,KARACHI,24.81,66.97,1103,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,Kandahar,31.613320,65.710130,577.128,204,KARACHI,24.81,66.97,767,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,Mazar-e Sharif,36.709040,67.110870,681.531,204,KARACHI,24.81,66.97,1325,NaN,NaN,NaN,NaN,NaN,NaN
5,Albania,Tiranë (Tirana),41.327500,19.818890,535.702,1075,BAR,42.08,19.08,104,NaN,NaN,NaN,NaN,NaN,NaN
6,Algeria,Annaba,36.900000,7.766670,379.346,104,CAGLIARI,39.20,9.17,284,NaN,NaN,NaN,NaN,NaN,NaN
7,Algeria,Batna,35.555970,6.174140,358.478,104,CAGLIARI,39.20,9.17,484,NaN,NaN,NaN,NaN,NaN,NaN
8,Algeria,Blida,36.480781,2.831943,535.641,1892,PALMA_DE_MALLORCA,39.55,2.64,342,NaN,NaN,NaN,NaN,NaN,NaN
9,Algeria,El Djazaïr (Algiers),36.752500,3.041970,3004.133,1892,PALMA_DE_MALLORCA,39.55,2.64,313,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# Get end datetime
end_dt = datetime.datetime.now()
# Calculate run timedelta
run_td = end_dt - start_dt
# Print timing information
print(f"Start:     {start_dt.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"End:       {end_dt.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Wall time: {run_td.seconds} s")

Start:     2025-02-05 18:35:07
End:       2025-02-05 18:35:41
Wall time: 34 s
